In [ ]:
import pandas as pd
import numpy as np
import urllib
import matplotlib.pyplot as plt
# url = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/datasets/housing/housing.csv"
# urllib.request.urlretrieve(url,"housing.csv")
housing = pd.read_csv("datasets/housing.csv");
housing.info()

In [ ]:
# EDA, scatterplot matrix
from pandas.plotting import scatter_matrix 
attrs=["housing_median_age","median_income","median_house_value","total_rooms"]
scatter_matrix(housing[attrs],figsize=(12,8));

In [ ]:
housing["rooms_per_household"]= housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"]= housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]= housing["population"]/housing["households"]

housing.drop("total_bedrooms",axis=1)
housing_num = housing.drop("ocean_proximity",axis=1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

housing_cat = housing[["ocean_proximity"]]
encoder = OneHotEncoder()
housing_cat_encoded = encoder.fit_transform(housing_cat)
housing_cat_encoded.toarray()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# custom transformer. 뭐하는거지?????
from sklearn.base import BaseEstimator, TransformerMixin
col_names = "total_rooms", "total_bedrooms", "population", "households"
# rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6
rooms_ix, bedrooms_ix, population_ix, households_ix = [
    housing.columns.get_loc(c) for c in col_names]
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    """
    """
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        pass

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"],
    index=housing.index)
housing_extra_attribs.head()

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]
full_pipeline = ColumnTransformer([
    # transformers from ("num",num_pipeline,num_attribs),
    ("imputer",SimpleImputer(strategy="median")),
    ("attribs_adder",CombinedAttributesAdder()),
    ("std_scaler",StandardScaler()),
    ("cat",OneHotEncoder(),cat_attribs)
])
housing_prepared = full_pipeline.fit_transform(housing_num, housing)

In [ ]:
# training with linear regression
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fig(housing_prepared,housing_labels)